In [1]:
import pandas as pd
import numpy as np

In [2]:
sales = pd.read_csv('sales_082020.csv', parse_dates=['date'])

sales.head()

,buyer_id,buyer_country,seller_id,seller_country,product_code,category,brand,currency,price,date
0,qzpjsi9t0o,GB,6oufpaot,GB,ydecnydr6obf,Dresses,NaN,GBP,19.11,2020-08-03
1,o79pns1qwo,GB,50xki2yg,GB,38chj507o6h2,Outerwear,Adidas Originals,GBP,15.92,2020-08-09
2,sjgbjdkhqx,GB,50xki2yg,GB,i9ynyxj5tdp8,Tops - Mens,NaN,GBP,6.37,2020-08-07
3,o79pns1qwo,GB,s0p322hh,GB,t50xe7moye8v,Tops - Mens,Berghaus,GBP,25.27,2020-08-09
4,o79pns1qwo,GB,dpkx192v,GB,hp4r9bjq68af,Bottoms - Womens,NaN,GBP,8.90,2020-08-09


In [3]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237671 entries, 0 to 237670
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   buyer_id        237671 non-null  object        
 1   buyer_country   237671 non-null  object        
 2   seller_id       237671 non-null  object        
 3   seller_country  237671 non-null  object        
 4   product_code    237671 non-null  object        
 5   category        237655 non-null  object        
 6   brand           119330 non-null  object        
 7   currency        237671 non-null  object        
 8   price           237671 non-null  float64       
 9   date            237671 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(8)
memory usage: 18.1+ MB


## 1. How many rows & columns in the sales dataset?

In [4]:
print(f"{sales.shape[0]} rows & {sales.shape[1]} columns in this dataset.")

237671 rows & 10 columns in this dataset.


## 2. What is the total number of sellers?

In [5]:
print(f"{sales['seller_id'].nunique()} sellers in dataset.")

114227 sellers in dataset.


## 3. What is the total value of sales in EUR?

In [8]:
currency_rates = pd.read_csv('currency_rates_082020_EUR.csv', parse_dates=['date'])

currency_rates.head()

,currency,date,rate
0,EUR,2020-08-01,1.0
1,EUR,2020-08-02,1.0
2,EUR,2020-08-03,1.0
3,EUR,2020-08-04,1.0
4,EUR,2020-08-05,1.0


In [9]:
currency_rates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   currency  220 non-null    object        
 1   date      220 non-null    datetime64[ns]
 2   rate      220 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 5.3+ KB


In [10]:
currency_rates['currency'].unique()

array(['EUR', 'GBP', 'USD', 'AUD', 'CAD', 'NZD', 'HKD', 'CHF', 'SEK',
       'DKK', 'ILS', 'PLN', 'THB', 'MXN', 'JPY', 'NOK', 'SGD', 'MYR',
       'TWD', 'HUF', 'CZK', 'BRL'], dtype=object)

In [14]:
sales['currency'].unique()

array(['GBP', 'USD', 'AUD', 'EUR', 'CAD', 'NZD', 'HKD', 'CHF', 'SEK',
       'DKK', 'ILS', 'PLN', 'THB', 'MXN', 'JPY', 'NOK', 'SGD', 'MYR',
       'TWD', 'HUF', 'CZK', 'BRL'], dtype=object)

In [11]:
currency_rates['date'].unique()

array(['2020-08-01T00:00:00.000000000', '2020-08-02T00:00:00.000000000',
       '2020-08-03T00:00:00.000000000', '2020-08-04T00:00:00.000000000',
       '2020-08-05T00:00:00.000000000', '2020-08-06T00:00:00.000000000',
       '2020-08-07T00:00:00.000000000', '2020-08-08T00:00:00.000000000',
       '2020-08-09T00:00:00.000000000', '2020-08-10T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [21]:
sales['date'].unique()

array(['2020-08-03T00:00:00.000000000', '2020-08-09T00:00:00.000000000',
       '2020-08-07T00:00:00.000000000', '2020-08-04T00:00:00.000000000',
       '2020-08-05T00:00:00.000000000', '2020-08-06T00:00:00.000000000',
       '2020-08-08T00:00:00.000000000'], dtype='datetime64[ns]')

In [12]:
sales_and_currency = sales.merge(currency_rates, how='left', on=['currency', 'date'])

In [20]:
sales_and_currency.isnull().any()["price"]

False

In [15]:
 sales_and_currency['price_€'] = (1 / sales_and_currency['rate']) * sales_and_currency['price']

In [16]:
print("Total sales are €{:,.0f}.".format(sales_and_currency['price_€'].sum()))

Total sales are €6,471,511.


## 4. Which brand has the highest number of purchases during the period?

In [17]:
(
    sales_and_currency
    .groupby('brand')
    .agg({'price_€': 'sum'})
    .sort_values(by='price_€', ascending=False)
    .round(2)
)

,price_€
brand,
Nike,385022.41
Adidas,114211.54
American Vintage,89939.91
Brandy Melville,87735.66
Dr. Martens,81491.62
...,...
butter LONDON,5.46
Ann Demeulemeester,5.26
Popular Sports,4.45


In [18]:
(
    sales_and_currency
    .groupby('brand')
    .agg({'price_€': 'sum'})
    .sort_values(by='price_€', ascending=False)
    .index[0]
)

'Nike'

## 5. How many items in the “Jewellery” category have no brand associated with them?

In [19]:
sales_and_currency[(sales_and_currency['category'] == 'Jewellery')\
                    & (sales_and_currency['brand'].isnull())]\
                    ['product_code'].nunique()

10931